In [2]:
import pandas as pd

# Load your time series data into a pandas dataframe
df = pd.read_csv('your_time_series_data.csv')

# Calculate the returns of your asset over a fixed lookback period
lookback_period = 10 # for example
df['returns'] = df['price'].pct_change(periods=lookback_period)

# Rank the assets based on their returns
df['rank'] = df['returns'].rank(ascending=False)

# Identify sequences of high momentum
high_momentum = df[df['rank'] <= 10] # for example

FileNotFoundError: [Errno 2] No such file or directory: 'your_time_series_data.csv'

### Graph the sequences

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

# Load your time series data into a pandas dataframe
df = pd.read_csv('your_time_series_data.csv')

# Calculate the returns of your asset over a fixed lookback period
lookback_period = 10 # for example
df['returns'] = df['price'].pct_change(periods=lookback_period)

# Rank the assets based on their returns
df['rank'] = df['returns'].rank(ascending=False)

# Identify sequences of high momentum
high_momentum = df[df['rank'] <= 10] # for example

# Plot the high momentum sequences
plt.plot(high_momentum['date'], high_momentum['price'])
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('High Momentum Sequences')
plt.show()


ModuleNotFoundError: No module named 'matplotlib'

In [4]:
import pandas as pd

# Load your time series data into a pandas dataframe
df = pd.read_csv('your_time_series_data.csv')

# Calculate the percentage change in price
df['pct_change'] = df['price'].pct_change()


FileNotFoundError: [Errno 2] No such file or directory: 'your_time_series_data.csv'

### Smoothing time series and detecting items outside of confidence interval

In [ ]:
import pandas as pd

# Load your time series data into a pandas dataframe
df = pd.read_csv('your_time_series_data.csv')

# Calculate the difference between two rows using diff()
df['diff'] = df['price'].diff()

# Calculate the moving average and standard deviation
rolling_mean = df['price'].rolling(window=10).mean()
rolling_std = df['price'].rolling(window=10).std()

# Identify the data points that are outside the confidence intervals
upper_band = rolling_mean + (2 * rolling_std)
lower_band = rolling_mean - (2 * rolling_std)
surges = df[(df['price'] > upper_band) | (df['price'] < lower_band)]
# From 

You can detect surges in price in a time series using various methods such as moving averages, standard deviations, and derivatives. One way to do this is to calculate the difference between two rows in your time series data using `diff()` and then identify the data points that are outside the confidence intervals based on moving average and standard deviation. Another way is to smooth the data using a Savitzky-Golay filter and then analyze the first (and second) derivative to determine a baseline. If you have the baseline identified, you can extract the signals¹.

Here is an example of how you can use `diff()` in Pandas to detect surges in price:

```python
import pandas as pd

# Load your time series data into a pandas dataframe
df = pd.read_csv('your_time_series_data.csv')

# Calculate the difference between two rows using diff()
df['diff'] = df['price'].diff()

# Calculate the moving average and standard deviation
rolling_mean = df['price'].rolling(window=10).mean()
rolling_std = df['price'].rolling(window=10).std()

# Identify the data points that are outside the confidence intervals
upper_band = rolling_mean + (2 * rolling_std)
lower_band = rolling_mean - (2 * rolling_std)
surges = df[(df['price'] > upper_band) | (df['price'] < lower_band)]
```

This code calculates the difference between two rows in your time series data using `diff()`, calculates the moving average and standard deviation of your price column using `rolling()`, and identifies the data points that are outside the confidence intervals by selecting the data points that are greater than or less than two standard deviations from the mean.

You can find more information about detecting surges in time series data in Python in this [article](https://stackoverflow.com/questions/59682415/detecting-rapid-increase-in-time-series) which covers various methods such as moving averages, standard deviations, and derivatives.

Source: Conversation with Bing, 7/24/2023
(1) python - Detecting rapid increase in time series - Stack Overflow. https://stackoverflow.com/questions/59682415/detecting-rapid-increase-in-time-series.
(2) Time Series Analysis in Python – A Comprehensive Guide with Examples. https://www.machinelearningplus.com/time-series/time-series-analysis-python/.
(3) Seasonality Detection with Fast Fourier Transform (FFT) and Python .... https://towardsai.net/p/machine-learning/seasonality-detection-with-fast-fourier-transform-fft-and-python.

In [ ]:
import matplotlib.pyplot as plt

# Plot the time series data
plt.plot(df['date'], df['price'])

# Plot the surges in price
plt.scatter(surges['date'], surges['price'], color='red')

plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Surges in Price')
plt.show()
